First we load `rpy2.ipython` extension so we can execute `R` code.

In [1]:
%load_ext rpy2.ipython

# Load up the Hadleyverse

In [2]:
%%R
library(dplyr)
library(tidyr)
library(ggplot2)


Attaching package: ‘dplyr’

The following object is masked from ‘package:stats’:

    filter

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



The data created by the experiment was logged in two places. Let's load them and join them.

In [3]:
%%R
src2ideology <- c('foxnews'=1,'msnbc'=-1,'cnn'=0)

Exp <- read.csv('webapp/experiment_data.csv', stringsAsFactors = FALSE) %>%
  mutate(select = as.numeric(summary != ''),
         source.ideology = src2ideology[source])

Survey <- read.csv('webapp/survey_data.csv', stringsAsFactors = FALSE) %>%
  filter(!is.na(response)) %>%
  group_by(userid) %>%
  summarise(response = first(response), reversed_scale = first(reversed_scale)) %>%
  mutate(subject.ideology = response - 3)

Results <- Exp %>% inner_join(Survey)

head(Results %>% select(userid, source.ideology, subject.ideology, select))

Joining by: "userid"
                                userid source.ideology subject.ideology select
1 066c7bae-cff5-4880-ba1a-558144238471              -1                0      1
2 066c7bae-cff5-4880-ba1a-558144238471              -1                0      1
3 066c7bae-cff5-4880-ba1a-558144238471               0                0      1
4 066c7bae-cff5-4880-ba1a-558144238471               0                0      1
5 066c7bae-cff5-4880-ba1a-558144238471              -1                0      0
6 066c7bae-cff5-4880-ba1a-558144238471              -1                0      0


Now we can tabulate the selections by source ideology to see if there was an ATE.

In [25]:
%%R
Results %>%
  group_by(source.ideology) %>%
  mutate(n = n()) %>%
  group_by(source.ideology, select) %>%
  summarise(k = n(), n = first(n), p = k / n)

Source: local data frame [6 x 5]
Groups: source.ideology

  source.ideology select   k   n         p
1              -1      0 250 334 0.7485030
2              -1      1  84 334 0.2514970
3               0      0 202 253 0.7984190
4               0      1  51 253 0.2015810
5               1      0 178 248 0.7177419
6               1      1  70 248 0.2822581
